In [1]:
#Import package pandas for data analysis
import pandas as pd
# Import package numpy for numeric computing
import numpy as np
from numpy import int64
from numpy import float64
from numpy import datetime64
# Import package matplotlib for visualisation/plotting
import matplotlib.pyplot as plt
import matplotlib.dates as dates
# Allows plots to appear directly in the notebook.
%matplotlib inline
# For dealing with some Accented characters (in Irish Place names)
import unidecode
# Date/time functionality
import datetime
import time
#for trigonometic calculations
import math
# Check if files exist
from os.path import exists
from os import makedirs
# System specific parameters and functions
import sys
import shutil
# look at some z-scores for inspecting outliers.
from scipy import stats
import seaborn as sb
# lookup lat/long and convert lat/long to national grid references.
#import geopy
#import pyproj

from patsy import dmatrices
from sklearn import metrics
#binary encoding by using the OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.linear_model import LinearRegression
from sklearn.tree import export_graphviz
#from sklearn.tree import export_text

from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split

#import graphviz
#from graphviz import Source
#to read all CSV files in a folder
import os
import glob
import pickle

#### Functions generated at notebooks that might be useful

In [2]:
#method to read chunks from a file and create a dataframe list.
def get_chunks(x):
    """x=[location, dtypes, datecolumns ,date_parser_func, cols]
    exp:mydateparser = lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S")
    exp:dtypes={'TRIPID' :'category','PROGRNUMBER':'int16','STOPPOINTID':'category','PLANNEDTIME_ARR':'int64','PLANNEDTIME_DEP':'int64','ACTUALTIME_ARR':'int64','ACTUALTIME_DEP':'int64','DIFFERENCETIME_ARR':'int64','DIFFERENCETIME_DEP':'int64','LINEID':'category','ROUTEID':'category','DIRECTION':'category','TRIPS_TIME_PROPORTION':'float32','TOTAL_JOURNEY_TIME':'int16','TIME_OF_DAY':'category','DAY_OF_WEEK':'category'}
    exp:cols=['Date time','Temperature','Wind Speed','Precipitation','Cloud Cover']
    exp:x=['/Chunks/DBUS/QP_Implementation',dtypes,['ACTUALTIME_ARR_DATETIME'],mydateparser, cols]"""
    cwd=os.getcwd()
    path = cwd+x[0]
    chunk_folder = glob.glob(os.path.join(path, "*.csv"))
    chunk_list=[]
    counter=0
    for filename in chunk_folder:
        # read the csv file
        if len(x)==4:
            df_chunk = pd.read_csv(filename, low_memory=False, dtype=x[1],parse_dates=x[2],date_parser=x[3])
        else:
            df_chunk = pd.read_csv(filename,usecols=x[4], low_memory=False, dtype=x[1],parse_dates=x[2],date_parser=x[3])
        chunk_list.append(df_chunk)
        print('Index at chunk_list',counter,' is ', filename, ': ',df_chunk.shape)
        counter+=1
    return chunk_list
#chunk_dbus_list=get_chunks(arg_list)

In [3]:
#method to parse dates
mydateparser = lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S")

In [4]:
#method that mathes list index in a dataframe list with id information in dataframe's column. 
#for this method to work id column should be constant. 
def id_list_index_matcher(df_list, column_name):
    df_id_list_index = pd.DataFrame()
    index_count=0
    for df in df_list:
        a=df[column_name].unique()
        df_id_list_index=df_id_list_index.append({'List_index':index_count,column_name:a[0]},ignore_index=True)
        index_count+=1
    return df_id_list_index

In [5]:
#methods that create csv files from dataframe list by naming them with unique value in id column.
#for this method to work id column should be constant. 
def write_to_csv(adress,df_list,id_column ):
    cwd=os.getcwd()
    path = cwd+adress
    for df in df_list:
        a=df[id_column].unique()
        file_name=a[0]
        saved_file_name=file_name+'.csv'
        my_file_bool=os.path.exists(path+saved_file_name)
        if my_file_bool==False:
            df.to_csv(path+saved_file_name,index=False)
            print('Location:', path+saved_file_name)

In [6]:
def write_to_csv_n_r_p(adress,df_list,id_column ):
    """without relative path input"""
    path = adress
    for df in df_list:
        a=df[id_column].unique()
        file_name=a[0]
        saved_file_name=file_name+'.csv'
        my_file_bool=os.path.exists(path+saved_file_name)
        if my_file_bool==False:
            df.to_csv(path+saved_file_name,index=False)
            print('Location:', path+saved_file_name)

In [7]:
#method that takes a dataframe list and prints list lenght and sum of all rows in all dataframes
def df_list_checker(dflist):
    print("number of df in list: ",len(dflist))
    row_count=0
    for df in dflist:
        row_count+=df.shape[0]
    print("number of rows of data in list: ", row_count)

In [8]:
#method to yield(return generator object) dfs for memory efficent concat( to avoid expotential memory use)
#https://stackoverflow.com/questions/70911236/how-can-i-concat-the-huge-pandas-dataframes-into-one
def yield_dfs(x):
    """x=[location, dtypes, datecolumns ,date_parser_func]
    exp:mydateparser = lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S")
    exp:dtypes={'TRIPID' :'category','PROGRNUMBER':'int16','STOPPOINTID':'category','PLANNEDTIME_ARR':'int64','PLANNEDTIME_DEP':'int64','ACTUALTIME_ARR':'int64','ACTUALTIME_DEP':'int64','DIFFERENCETIME_ARR':'int64','DIFFERENCETIME_DEP':'int64','LINEID':'category','ROUTEID':'category','DIRECTION':'category','TRIPS_TIME_PROPORTION':'float32','TOTAL_JOURNEY_TIME':'int16','TIME_OF_DAY':'category','DAY_OF_WEEK':'category'}
    exp:x=['/Chunks/DBUS/QP_Implementation',dtypes,['ACTUALTIME_ARR_DATETIME'],mydateparser]"""
    cwd=os.getcwd()
    path = cwd+x[0]
    file_list = glob.glob(os.path.join(path, "*.csv"))
    """generator function to yield dataframes."""
    for filename in file_list:
        df = pd.read_csv(filename, low_memory=False, dtype=x[1],parse_dates=x[2],date_parser=x[3],index_col=False)
        print(df.shape)
        yield df

#df_dbus = pd.concat(yield_dfs(x), axis=0,ignore_index=True)

In [9]:
#method that takes a directory and reads all files inside that directory and its subdirectories 
#creates a dataframe from them and add to dataframe list
def get_chunks_subdirectory(location):
    cwd=os.getcwd()
    adress=location
    path = cwd+adress
    chunk_list=[]
    for root, dirnames, files in os.walk(path):
        print(dirnames)
        print('='*66)
        for mfile in files:
            # read the csv file
            df_chunk = pd.read_csv(os.path.join(root, mfile), low_memory=False)
            chunk_list.append(df_chunk)
            print('Location:', mfile)
    return chunk_list
#chunk_dbus_list_v2=get_chunks_subdirectory('/Chunks/DBUS/LINEID_CHUNKS/')

In [10]:
def write_to_csv_if_in_list(adress,df_list,id_column,IMP_List ):
    cwd=os.getcwd()
    path = cwd+adress
    for df in df_list:
        a=df[id_column].unique()
        if a in IMP_List:
            file_name=a[0]
            saved_file_name=file_name+'.csv'
            my_file_bool=os.path.exists(path+saved_file_name)
            if my_file_bool==False:
                df.to_csv(path+saved_file_name,index=False)
                print('Location:', path+saved_file_name)

In [11]:
def get_sourcefeatures(file, ROUTEID):
    df = pd.read_csv(file, low_memory=False)
    targetrow_df= df[df.ROUTEID == ROUTEID ]
    features_list=[]
    for column in targetrow_df.columns.values:
        if targetrow_df.iloc[0][column]==1:
           features_list.append(column)
    features_list.remove('TOTAL_JOURNEY_TIME')
    return features_list

In [12]:
# Define a Helper function to compute/print all our metrics
def printMetrics(actualVal, predictions):
    #classification evaluation measures
    print('\n==============================================================================')
    print("MAE: ", metrics.mean_absolute_error(actualVal, predictions))
    print("MSE: ", metrics.mean_squared_error(actualVal, predictions))
    print("RMSE: ", metrics.mean_squared_error(actualVal, predictions)**0.5)
    print("R²: ", metrics.r2_score(actualVal, predictions))

Adresses

In [13]:
home_path=os.path.normpath(os.getcwd() + os.sep + os.pardir)
home_path

'/home/team8/notebook'

In [14]:
KNNR_results_adress=home_path+ '/Pelin/KNNR_results.csv'
RFR_results_adress=home_path+'/Pelin/RFR_results.csv'
LR_results_adress=home_path+'/Lorna/LinearRegression/LR_results.csv'
SVR_results_adress=home_path+'/Lorna/SupportVectorReg/SVR_results.csv'

Read all results in to dataframe and add them to a list

In [15]:
df_KNNR_results = pd.read_csv(KNNR_results_adress )
df_RFR_results= pd.read_csv(RFR_results_adress)
df_LR_results=pd.read_csv(LR_results_adress)
df_SVR_results=pd.read_csv(SVR_results_adress)
result_dfs_list=[df_KNNR_results,df_RFR_results,df_LR_results,df_SVR_results]

In [16]:
ML_all_results_df=pd.concat(result_dfs_list,axis=0,ignore_index=True)
ML_all_results_df

,MODEL,ROUTEID,MAE,RMSE,R2,Unnamed: 0
0,KNNR,18_3,-624.053202,-866.656039,0.409486,NaN
1,KNNR,67_6,-555.853574,-749.716890,0.353909,NaN
2,KNNR,142_9,-228.863360,-403.304348,0.014865,NaN
3,KNNR,17_16,-260.408106,-374.638036,0.504578,NaN
4,KNNR,46A_74,-688.958417,-923.229979,0.173054,NaN
...,...,...,...,...,...,...
1215,SVR,17A_14,-492.526946,-873.644221,-0.078849,NaN
1216,SVR,66A_38,-572.360802,-898.557907,0.082264,NaN
1217,SVR,15A_85,-638.280129,-887.832880,0.037202,NaN
1218,SVR,47_136,-677.557415,-1035.238790,0.040795,NaN


In [17]:
ML_best_results_series=ML_all_results_df.groupby(['ROUTEID'])['R2'].max()
ML_best_results_series.sort_values().head(5)

ROUTEID
111_9    -0.026251
53_23    -0.023458
185_59   -0.020298
47_135   -0.014988
185_51   -0.010153
Name: R2, dtype: float64

In [18]:
ML_best_results_series_below_treshold=ML_best_results_series[ML_best_results_series <0.2]
print("Below 0.2 R2 treshold: ",ML_best_results_series_below_treshold.value_counts().sum())
ML_best_results_series_negative_R2=ML_best_results_series[ML_best_results_series <0]
print("Negative R2: ",ML_best_results_series_negative_R2.value_counts().sum())

Below 0.2 R2 treshold:  27
Negative R2:  5


In [19]:
ML_routes_below_treshold_R2_df=pd.DataFrame({'R2':ML_best_results_series_below_treshold}).reset_index()

In [20]:
ML_routes_below_treshold_R2_df.to_csv('ML_route_bel_tres_R2_cor01_df.csv',index=False)

#### Investigating reason for routes falling behind treshold in all 4 model 

In [21]:
cols=['TRIPID','PROGRNUMBER','PLANNEDTIME_DEP','ROUTEID','ACTUALTIME_ARR_DATETIME','TOTAL_JOURNEY_TIME','TIME_OF_DAY','DAY_OF_WEEK','TRIPS_TIME_PROPORTION_v2','PLANNEDTIME_DEP_R_M5','rain','temp','wet_bulb_temp(C)','dew_pt_temp(C)','vapour_pressure(hPa)','humidity(%)','sea_lvl_pressure(hPa)','sin_hour_of_day','cos_hour_of_day','(Friday,)','(Monday,)','(Saturday,)','(Sunday,)','(Thursday,)','(Tuesday,)','(Wednesday,)']
dtypes={'TRIPID' :'category','PROGRNUMBER':'int16','PLANNEDTIME_DEP':'int64','ROUTEID':'category','TOTAL_JOURNEY_TIME':'int16','TIME_OF_DAY':'category','DAY_OF_WEEK':'category','TRIPS_TIME_PROPORTION_v2':'float32','PLANNEDTIME_DEP_R_M5':'category','rain':'float16','temp':'float16','wet_bulb_temp(C)':'float16','dew_pt_temp(C)':'float16','vapour_pressure(hPa)':'float16','humidity(%)':'int16','sea_lvl_pressure(hPa)':'float16','sin_hour_of_day':'float16','cos_hour_of_day':'float16','(Friday,)':'int16','(Monday,)':'int16','(Saturday,)':'int16','(Sunday,)':'int16','(Thursday,)':'int16','(Tuesday,)':'int16','(Wednesday,)':'int16'}
x=['/Chunks/DBUS/Predictive_Model_Chunks',dtypes,['ACTUALTIME_ARR_DATETIME'],mydateparser]

In [22]:
dbus_df_list_v3=get_chunks(x)

Index at chunk_list 0  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/18_3.csv :  (3896, 26)
Index at chunk_list 1  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/67_6.csv :  (4001, 26)
Index at chunk_list 2  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/142_9.csv :  (252, 24)
Index at chunk_list 3  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/17_16.csv :  (307, 23)
Index at chunk_list 4  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/46A_74.csv :  (11186, 26)
Index at chunk_list 5  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/122_14.csv :  (4164, 26)
Index at chunk_list 6  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/150_8.csv :  (4907, 26)
Index at chunk_list 7  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/33X_49.csv :  (181, 24)
Index at chunk_list 8  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks

Index at chunk_list 79  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/14_16.csv :  (5208, 26)
Index at chunk_list 80  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/15A_83.csv :  (5097, 26)
Index at chunk_list 81  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/39_21.csv :  (4147, 26)
Index at chunk_list 82  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/151_18.csv :  (831, 26)
Index at chunk_list 83  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/238_15.csv :  (1521, 26)
Index at chunk_list 84  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/79A_27.csv :  (2485, 26)
Index at chunk_list 85  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/7_73.csv :  (781, 26)
Index at chunk_list 86  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/69_47.csv :  (592, 25)
Index at chunk_list 87  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Mo

Index at chunk_list 153  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/47_139.csv :  (1842, 26)
Index at chunk_list 154  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/41C_84.csv :  (349, 26)
Index at chunk_list 155  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/185_53.csv :  (519, 26)
Index at chunk_list 156  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/9_5.csv :  (6631, 26)
Index at chunk_list 157  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/4_12.csv :  (4046, 26)
Index at chunk_list 158  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/27B_36.csv :  (169, 24)
Index at chunk_list 159  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/1_40.csv :  (4834, 26)
Index at chunk_list 160  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/69_44.csv :  (1267, 26)
Index at chunk_list 161  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predict

Index at chunk_list 226  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/69_45.csv :  (530, 25)
Index at chunk_list 227  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/41_22.csv :  (373, 26)
Index at chunk_list 228  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/17A_15.csv :  (1710, 24)
Index at chunk_list 229  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/185_51.csv :  (229, 24)
Index at chunk_list 230  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/68_66.csv :  (197, 26)
Index at chunk_list 231  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/185_52.csv :  (392, 25)
Index at chunk_list 232  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/123_34.csv :  (7069, 26)
Index at chunk_list 233  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks/25D_275.csv :  (208, 24)
Index at chunk_list 234  is  /home/team8/notebook/Pelin/Chunks/DBUS/Pred

1. find out sample size of routes which fall below treshold

In [23]:
Below_treshold_routes_df=pd.DataFrame()
for ROUTEID in ML_routes_below_treshold_R2_df['ROUTEID'].values:
    for df in dbus_df_list_v3:
        if df['ROUTEID'].unique()[0]==ROUTEID:
            Below_treshold_routes_df=pd.concat([Below_treshold_routes_df,df],axis=0,ignore_index=True)

In [24]:
Below_treshold_routes_df.shape

(13674, 26)

In [25]:
total_dataset_row_count=0
for df in dbus_df_list_v3:
    total_dataset_row_count+=df.shape[0]
print(total_dataset_row_count)
print(Below_treshold_routes_df.shape[0]/total_dataset_row_count, " below treshold data is only this much percentage of total data")

631577
0.021650566755914164  below treshold data is only this much percentage of total data


#### Concensus on course of action: we will drop those 27 rows from training datasets and update static tables for backend to exclude them

In [26]:
cols=['ROUTEID','TOTAL_JOURNEY_TIME','PLANNEDTIME_DEP','rain','temp','wet_bulb_temp(C)','vapour_pressure(hPa)','humidity(%)','sea_lvl_pressure(hPa)','sin_hour_of_day','cos_hour_of_day','friday','monday','saturday','sunday','thursday','tuesday','wednesday']
dtypes={'PLANNEDTIME_DEP':'int64','ROUTEID':'category','TOTAL_JOURNEY_TIME':'int16','rain':'float16','temp':'float16','wet_bulb_temp(C)':'float16','vapour_pressure(hPa)':'float16','humidity(%)':'int16','sea_lvl_pressure(hPa)':'float16','sin_hour_of_day':'float16','cos_hour_of_day':'float16','friday':'int16','monday':'int16','saturday':'int16','sunday':'int16','thursday':'int16','tuesday':'int16','wednesday':'int16'}
x=['/Chunks/DBUS/Predictive_Model_Chunks_backup',dtypes,[],mydateparser]

In [27]:
dbus_model_dfs_list=get_chunks(x)

Index at chunk_list 0  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/18_3.csv :  (3896, 19)
Index at chunk_list 1  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/67_6.csv :  (4001, 19)
Index at chunk_list 2  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/142_9.csv :  (252, 17)
Index at chunk_list 3  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/17_16.csv :  (307, 16)
Index at chunk_list 4  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/46A_74.csv :  (11186, 19)
Index at chunk_list 5  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/122_14.csv :  (4164, 19)
Index at chunk_list 6  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/150_8.csv :  (4907, 19)
Index at chunk_list 7  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/33X_49.csv :  (181, 17)
Index at chunk_list 8  is  /home/

Index at chunk_list 79  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/14_16.csv :  (5208, 19)
Index at chunk_list 80  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/15A_83.csv :  (5097, 19)
Index at chunk_list 81  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/39_21.csv :  (4147, 19)
Index at chunk_list 82  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/151_18.csv :  (831, 19)
Index at chunk_list 83  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/238_15.csv :  (1521, 19)
Index at chunk_list 84  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/79A_27.csv :  (2485, 19)
Index at chunk_list 85  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/7_73.csv :  (781, 19)
Index at chunk_list 86  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/69_47.csv :  (592, 18)
Index at chunk_list 87  

Index at chunk_list 149  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/239_26.csv :  (910, 19)
Index at chunk_list 150  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/13_65.csv :  (548, 19)
Index at chunk_list 151  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/84X_57.csv :  (221, 17)
Index at chunk_list 152  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/54A_11.csv :  (2786, 19)
Index at chunk_list 153  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/47_139.csv :  (1842, 19)
Index at chunk_list 154  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/41C_84.csv :  (349, 19)
Index at chunk_list 155  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/185_53.csv :  (519, 19)
Index at chunk_list 156  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/9_5.csv :  (6631, 19)
Index at chunk_li

Index at chunk_list 218  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/68_78.csv :  (242, 18)
Index at chunk_list 219  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/66A_34.csv :  (653, 19)
Index at chunk_list 220  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/27_12.csv :  (196, 18)
Index at chunk_list 221  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/7A_85.csv :  (2786, 19)
Index at chunk_list 222  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/33_70.csv :  (1238, 19)
Index at chunk_list 223  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/238_11.csv :  (1522, 19)
Index at chunk_list 224  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/63_25.csv :  (782, 19)
Index at chunk_list 225  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/104_16.csv :  (1011, 19)
Index at chunk_li

Index at chunk_list 297  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/63_26.csv :  (1453, 19)
Index at chunk_list 298  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/66B_58.csv :  (859, 19)
Index at chunk_list 299  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/17_10.csv :  (2066, 18)
Index at chunk_list 300  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/17A_14.csv :  (422, 16)
Index at chunk_list 301  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/66A_38.csv :  (947, 19)
Index at chunk_list 302  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/15A_85.csv :  (1058, 19)
Index at chunk_list 303  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/47_136.csv :  (1832, 19)
Index at chunk_list 304  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/40D_103.csv :  (2045, 19)


In [28]:
dbus_model_dfs_list[0].head(5)

,PLANNEDTIME_DEP,ROUTEID,TOTAL_JOURNEY_TIME,rain,temp,wet_bulb_temp(C),dew_pt_temp(C),vapour_pressure(hPa),humidity(%),sea_lvl_pressure(hPa),sin_hour_of_day,cos_hour_of_day,friday,monday,saturday,sunday,thursday,tuesday,wednesday
0,37010,18_3,2473,0.000000,5.601562,4.300781,2.3,7.199219,79,993.0,0.500000,-0.866211,0,1,0,0,0,0,0
1,41726,18_3,3135,0.000000,7.500000,5.601562,3.0,7.601562,73,995.0,0.258789,-0.965820,0,1,0,0,0,0,0
2,45238,18_3,2446,0.000000,7.500000,5.601562,3.0,7.601562,73,995.0,0.000000,-1.000000,0,1,0,0,0,0,0
3,47426,18_3,3137,0.199951,7.300781,5.898438,3.9,8.101562,78,996.0,-0.258789,-0.965820,0,1,0,0,0,0,0
4,49711,18_3,3307,0.099976,6.398438,5.199219,3.5,7.800781,81,998.5,-0.258789,-0.965820,0,1,0,0,0,0,0


In [49]:
extracted_routeid_count=0
dbus_model_dfs_list_v2=[]
route_ids_tobe_dropped=ML_routes_below_treshold_R2_df['ROUTEID'].values
for df in dbus_model_dfs_list:
    routeid=df['ROUTEID'].unique()[0]
    if not routeid in route_ids_tobe_dropped:
        dbus_model_dfs_list_v2.append(df)
        dbus_model_id_list_v2.append(routeid)
    else:
        print("Excluding ", routeid ," from model training dataframes list")
        extracted_routeid_count+=1
print(extracted_routeid_count , " df extracted from model training dataframes list")

Excluding  185_57  from model training dataframes list
Excluding  68_79  from model training dataframes list
Excluding  83_25  from model training dataframes list
Excluding  84A_3  from model training dataframes list
Excluding  185_59  from model training dataframes list
Excluding  27B_40  from model training dataframes list
Excluding  40D_101  from model training dataframes list
Excluding  185_60  from model training dataframes list
Excluding  44B_61  from model training dataframes list
Excluding  33X_46  from model training dataframes list
Excluding  13_73  from model training dataframes list
Excluding  84X_57  from model training dataframes list
Excluding  185_53  from model training dataframes list
Excluding  53_23  from model training dataframes list
Excluding  68_85  from model training dataframes list
Excluding  33B_58  from model training dataframes list
Excluding  47_135  from model training dataframes list
Excluding  7B_91  from model training dataframes list
Excluding  185_5

In [50]:
df_no_dbus_model_dfs_list_v2=0
for df in dbus_model_dfs_list_v2:
    df_no_dbus_model_dfs_list_v2+=1
df_no_dbus_model_dfs_list_v2

278

In [51]:
write_to_csv('/Chunks/DBUS/Predictive_Model_Chunks_backup/',dbus_model_dfs_list_v2,'ROUTEID' )

Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/18_3.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/67_6.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/142_9.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/17_16.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/46A_74.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/122_14.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/150_8.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/33X_49.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/83_19.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/270_44.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/84_30.csv
Location

Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/122_16.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/239_28.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/38A_20.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/40D_104.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/53_21.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/102_9.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/38B_40.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/26_29.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/114_6.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/84X_62.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/7A_86.csv
Loc

Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/33A_25.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/76_36.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/25_269.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/150_9.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/220_12.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/67X_46.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/130_11.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/33A_22.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/122_18.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/68_78.csv
Location: /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/66A_34.csv
Lo